# Query Data API - Tutorial

This tutorial shows you how to use the Agenta Query Data API to retrieve and analyze your LLM traces. You'll learn how to:

- Set up the API client with authentication
- Query spans and traces with filters
- Filter by attributes, time ranges, and status codes
- Use advanced filters with logical operators
- Analyze trace data to calculate costs and latencies

## What You'll Build

We'll create scripts that:
1. Query recent traces from your applications
2. Filter traces by type, status, and custom attributes
3. Analyze cost and performance metrics
4. Find problematic traces (errors, slow responses)
5. Export trace data for further analysis

## Install Dependencies

In [ ]:
pip install -U requests pandas

## Setup

Before using the API, you need your Agenta API key. You can create API keys from the Settings page in your Agenta workspace.

In [ ]:
import os

os.environ["AGENTA_HOST"] = (
    "https://cloud.agenta.ai"  # Default value, change for self-hosted
)
os.environ["AGENTA_API_KEY"] = ""

In [ ]:
import os
import requests
from getpass import getpass
from datetime import datetime, timedelta, timezone

# Get API credentials
AGENTA_HOST = os.getenv("AGENTA_HOST", "https://cloud.agenta.ai")
api_key = os.getenv("AGENTA_API_KEY")
if not api_key:
    api_key = getpass("Enter your Agenta API key: ")
    os.environ["AGENTA_API_KEY"] = api_key

# Setup base configuration
BASE_URL = f"{AGENTA_HOST}/api/preview/tracing/spans/query"
HEADERS = {"Authorization": f"ApiKey {api_key}", "Content-Type": "application/json"}

print(f"✓ Connected to {AGENTA_HOST}")

## Part 1: Query Recent Traces

Let's start by querying traces from the last 7 days. We'll use the `focus=trace` parameter to get complete trace trees instead of individual spans.

In [ ]:
# Query traces from the last 7 days
now = datetime.now(timezone.utc)
week_ago = now - timedelta(days=7)

query = {
    "focus": "trace",
    "oldest": week_ago.isoformat(),
    "newest": now.isoformat(),
    "limit": 5,
}

response = requests.post(BASE_URL, headers=HEADERS, json=query)
data = response.json()

print(f"Found {data['count']} traces")
print(f"Trace IDs: {list(data.get('traces', {}).keys())}")

## Part 2: Query Spans with Filters

Now let's query individual spans and filter by type. We'll look for LLM spans specifically.

In [ ]:
# Query LLM spans
query = {
    "focus": "span",
    "limit": 10,
    "filter": {
        "operator": "and",
        "conditions": [
            {
                "field": "attributes",
                "key": "ag.type.span",
                "operator": "is",
                "value": "llm",
            }
        ],
    },
}

response = requests.post(BASE_URL, headers=HEADERS, json=query)
data = response.json()

print(f"Found {data['count']} LLM spans")

# Display first span details
if data.get("spans"):
    span = data["spans"][0]
    print("\nFirst span:")
    print(f"  Name: {span.get('span_name')}")
    print(f"  Status: {span.get('status_code')}")
    print(f"  Start: {span.get('start_time')}")
    print(f"  End: {span.get('end_time')}")

## Part 3: Filter by Status Code

Let's find traces that encountered errors. This is useful for debugging and monitoring application health.

In [ ]:
# Find error traces
query = {
    "focus": "trace",
    "limit": 10,
    "filter": {
        "operator": "and",
        "conditions": [
            {"field": "status_code", "operator": "is", "value": "STATUS_CODE_ERROR"}
        ],
    },
}

response = requests.post(BASE_URL, headers=HEADERS, json=query)
data = response.json()

print(f"Found {data['count']} traces with errors")

if data.get("traces"):
    for trace_id in list(data["traces"].keys())[:3]:
        print(f"\nError trace: {trace_id}")

## Part 4: Advanced Filtering with Multiple Conditions

Let's use multiple filters to find specific traces. We'll look for successful LLM calls that took longer than 2 seconds.

In [ ]:
# Find slow but successful LLM calls
query = {
    "focus": "span",
    "limit": 10,
    "filter": {
        "operator": "and",
        "conditions": [
            {
                "field": "attributes",
                "key": "ag.type.span",
                "operator": "is",
                "value": "llm",
            },
            {
                "field": "status_code",
                "operator": "is_not",
                "value": "STATUS_CODE_ERROR",
            },
            {
                "field": "attributes",
                "key": "ag.metrics.unit.duration",
                "operator": "gt",
                "value": 2000,  # milliseconds
            },
        ],
    },
}

response = requests.post(BASE_URL, headers=HEADERS, json=query)
data = response.json()

print(f"Found {data['count']} slow LLM spans (>2s)")

if data.get("spans"):
    for span in data["spans"][:3]:
        duration = (
            span.get("attributes", {})
            .get("ag", {})
            .get("metrics", {})
            .get("duration", {})
            .get("cumulative", "N/A")
        )
        print(f"  {span.get('span_name')}: {duration}ms")

## Part 5: Nested Logical Operators

Let's create a more complex query using nested logical operators. We'll find spans that are either errors OR slow responses.

In [ ]:
# Find problematic spans (errors OR slow)
query = {
    "focus": "span",
    "limit": 10,
    "filter": {
        "operator": "and",
        "conditions": [
            {
                "field": "attributes",
                "key": "ag.type.span",
                "operator": "is",
                "value": "llm",
            },
            {
                "operator": "or",
                "conditions": [
                    {"field": "status_code", "value": "STATUS_CODE_ERROR"},
                    {
                        "field": "attributes",
                        "key": "ag.metrics.unit.duration",
                        "operator": "gt",
                        "value": 5000,
                    },
                ],
            },
        ],
    },
}

response = requests.post(BASE_URL, headers=HEADERS, json=query)
data = response.json()

print(f"Found {data['count']} problematic spans")

if data.get("spans"):
    for span in data["spans"][:5]:
        status = span.get("status_code")
        duration = (
            span.get("attributes", {})
            .get("ag", {})
            .get("metrics", {})
            .get("duration", {})
            .get("cumulative", "N/A")
        )
        print(f"  {span.get('span_name')}: Status={status}, Duration={duration}ms")

## Part 6: Analyze Cost and Token Usage

Let's query LLM spans and analyze their costs and token usage. This helps you understand your LLM spending.

In [ ]:
# Query LLM spans with cost tracking
query = {
    "focus": "span",
    "limit": 50,
    "filter": {
        "operator": "and",
        "conditions": [
            {
                "field": "attributes",
                "key": "ag.type.span",
                "operator": "is",
                "value": "llm",
            },
            {
                "field": "attributes",
                "key": "ag.metrics.unit.cost",
                "operator": "exists",
            },
        ],
    },
}

response = requests.post(BASE_URL, headers=HEADERS, json=query)
data = response.json()

print(f"Analyzing {data['count']} LLM spans with cost data\n")

if data.get("spans"):
    total_cost = 0
    total_tokens = 0
    total_duration = 0

    for span in data["spans"]:
        metrics = span.get("attributes", {}).get("ag", {}).get("metrics", {})

        # Extract cost
        cost = metrics.get("costs", {}).get("cumulative", {}).get("total", 0)
        total_cost += cost

        # Extract tokens
        tokens = metrics.get("tokens", {}).get("cumulative", {}).get("total", 0)
        total_tokens += tokens

        # Extract duration
        duration = metrics.get("duration", {}).get("cumulative", 0)
        total_duration += duration

    print("Summary:")
    print(f"  Total Cost: ${total_cost:.4f}")
    print(f"  Total Tokens: {total_tokens:,}")
    print(f"  Average Cost per Span: ${(total_cost / len(data['spans'])):.4f}")
    print(f"  Average Tokens per Span: {int(total_tokens / len(data['spans']))}")
    print(f"  Average Duration: {int(total_duration / len(data['spans']))}ms")

## Part 7: Filter by Span Name Pattern

Let's use string matching operators to find specific types of operations. We'll search for OpenAI-related spans.

In [ ]:
# Find OpenAI spans using pattern matching
query = {
    "focus": "span",
    "limit": 10,
    "filter": {
        "operator": "and",
        "conditions": [
            {"field": "span_name", "operator": "contains", "value": "openai"}
        ],
    },
}

response = requests.post(BASE_URL, headers=HEADERS, json=query)
data = response.json()

print(f"Found {data['count']} OpenAI spans")

if data.get("spans"):
    span_names = set(span.get("span_name") for span in data["spans"])
    print("\nUnique span names:")
    for name in span_names:
        print(f"  - {name}")

## Part 8: Export Trace Data to DataFrame

Let's export our trace data to a pandas DataFrame for further analysis and visualization.

In [ ]:
import pandas as pd

# Query recent spans
query = {
    "focus": "span",
    "limit": 100,
    "filter": {
        "operator": "and",
        "conditions": [
            {
                "field": "attributes",
                "key": "ag.type.span",
                "operator": "is",
                "value": "llm",
            }
        ],
    },
}

response = requests.post(BASE_URL, headers=HEADERS, json=query)
data = response.json()

# Convert to DataFrame
records = []
for span in data.get("spans", []):
    metrics = span.get("attributes", {}).get("ag", {}).get("metrics", {})

    record = {
        "trace_id": span.get("trace_id"),
        "span_id": span.get("span_id"),
        "span_name": span.get("span_name"),
        "status": span.get("status_code"),
        "start_time": span.get("start_time"),
        "end_time": span.get("end_time"),
        "duration_ms": metrics.get("duration", {}).get("cumulative", 0),
        "cost": metrics.get("costs", {}).get("cumulative", {}).get("total", 0),
        "total_tokens": metrics.get("tokens", {}).get("cumulative", {}).get("total", 0),
        "prompt_tokens": metrics.get("tokens", {})
        .get("cumulative", {})
        .get("prompt", 0),
        "completion_tokens": metrics.get("tokens", {})
        .get("cumulative", {})
        .get("completion", 0),
    }
    records.append(record)

df = pd.DataFrame(records)

print(f"Created DataFrame with {len(df)} rows\n")
print("First 5 rows:")
print(df.head())

print("\nBasic statistics:")
print(df[["duration_ms", "cost", "total_tokens"]].describe())

## Part 9: Time-Based Analysis

Let's analyze how your costs and latencies change over time.

In [ ]:
# Convert timestamps to datetime
df["start_time"] = pd.to_datetime(df["start_time"])
df["end_time"] = pd.to_datetime(df["end_time"])

# Group by hour
df["hour"] = df["start_time"].dt.floor("H")
hourly_stats = (
    df.groupby("hour")
    .agg(
        {
            "span_id": "count",
            "duration_ms": "mean",
            "cost": "sum",
            "total_tokens": "sum",
        }
    )
    .rename(columns={"span_id": "num_calls"})
)

print("Hourly statistics:")
print(hourly_stats)

print(f"\nPeak usage hour: {hourly_stats['num_calls'].idxmax()}")
print(
    f"Highest cost hour: {hourly_stats['cost'].idxmax()} (${hourly_stats['cost'].max():.4f})"
)

## Part 10: Filter by Time Range

Let's query traces from a specific time window. This is useful for analyzing specific incidents or time periods.

In [ ]:
# Query last 24 hours
now = datetime.now(timezone.utc)
yesterday = now - timedelta(days=1)

query = {
    "focus": "span",
    "oldest": yesterday.isoformat(),
    "newest": now.isoformat(),
    "limit": 100,
    "filter": {
        "operator": "and",
        "conditions": [
            {
                "field": "attributes",
                "key": "ag.type.span",
                "operator": "is",
                "value": "llm",
            }
        ],
    },
}

response = requests.post(BASE_URL, headers=HEADERS, json=query)
data = response.json()

print(f"Last 24 hours: {data['count']} LLM spans")

if data.get("spans"):
    # Calculate totals
    total_cost = sum(
        span.get("attributes", {})
        .get("ag", {})
        .get("metrics", {})
        .get("costs", {})
        .get("cumulative", {})
        .get("total", 0)
        for span in data["spans"]
    )

    error_count = sum(
        1 for span in data["spans"] if span.get("status_code") == "STATUS_CODE_ERROR"
    )

    print("\nSummary for last 24 hours:")
    print(f"  Total Cost: ${total_cost:.4f}")
    print(f"  Error Rate: {(error_count / len(data['spans']) * 100):.2f}%")
    print(
        f"  Success Rate: {((len(data['spans']) - error_count) / len(data['spans']) * 100):.2f}%"
    )

## Summary

In this tutorial, you learned how to:

1. ✓ Set up the Agenta Query Data API client
2. ✓ Query traces and spans with filters
3. ✓ Filter by attributes, status codes, and time ranges
4. ✓ Use advanced filters with logical operators
5. ✓ Analyze cost and performance metrics
6. ✓ Export trace data to pandas DataFrames
7. ✓ Perform time-based analysis

## Next Steps

- Learn about the [Analytics Data API](/observability/query-data/analytics-data) for aggregated metrics
- Explore [filtering in the UI](/observability/using-the-ui/filtering-traces) for visual query building
- Check out [trace annotations](/observability/trace-with-python-sdk/annotate-traces) for adding feedback data
- Read the complete [API reference](/reference/api) for all available endpoints